In [2]:
%load_ext autoreload

In [ ]:
%load_ext blackcellmagic

In [3]:
%autoreload 2

In [4]:
from prefect import Flow, Parameter, unmapped
import pandas as pd
from prefect.engine.executors import DaskExecutor
from meta_model import MetaModel

In [5]:
input_df = pd.read_csv("../data/titanic/train.csv")
test= pd.read_csv("../data/titanic/test.csv")

In [10]:
from ml_flow import data_cleaning_flow
executor = DaskExecutor()
data_cleaner = data_cleaning_flow.run(
    input_data= input_df, 
    problem="binary classification", 
    target = "Survived", 
    features = "infer",
    executor=executor
)

[2020-04-13 00:02:37,088] INFO - prefect.FlowRunner | Beginning Flow run for 'data_cleaning'
[2020-04-13 00:02:37,093] INFO - prefect.FlowRunner | Starting flow run.
[2020-04-13 00:02:37,182] INFO - prefect.TaskRunner | Task 'features': Starting task run...
[2020-04-13 00:02:37,191] INFO - prefect.TaskRunner | Task 'input_data': Starting task run...
[2020-04-13 00:02:37,210] INFO - prefect.TaskRunner | Task 'features': finished task run for task with final state: 'Success'
[2020-04-13 00:02:37,245] INFO - prefect.TaskRunner | Task 'input_data': finished task run for task with final state: 'Success'
[2020-04-13 00:02:37,291] INFO - prefect.TaskRunner | Task 'target': Starting task run...
[2020-04-13 00:02:37,297] INFO - prefect.TaskRunner | Task 'recreate_tinydb': Starting task run...
[2020-04-13 00:02:37,326] INFO - prefect.TaskRunner | Task 'problem': Starting task run...
[2020-04-13 00:02:37,354] INFO - prefect.TaskRunner | Task 'target': finished task run for task with final state: 

In [31]:
dir(flow_state)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slotnames__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_result',
 'cached_inputs',
 'children',
 'color',
 'context',
 'deserialize',
 'is_cached',
 'is_failed',
 'is_finished',
 'is_looped',
 'is_mapped',
 'is_meta_state',
 'is_pending',
 'is_queued',
 'is_retrying',
 'is_running',
 'is_scheduled',
 'is_skipped',
 'is_submitted',
 'is_successful',
 'message',
 'parents',
 'result',
 'serialize']

In [20]:
with Flow("data_visualization") as flow:
    transformed_train_df = pd.read_feather("transformed_train.df")
    transformed_valid_df = pd.read_feather("transformed_valid.df")
    svd = fit_svd(transformed_train_df)
    svd_train = svd_transform(svd, transformed_train_df, "transformed_train_df",tinydb)
    svd_valid = svd_transform(svd, transformed_valid_df, "transformed_valid_df",tinydb)
    

In [21]:
with Flow("meta_model") as flow:
    transformed_train_df = pd.read_feather("transformed_train.df")
    transformed_valid_df = pd.read_feather("transformed_valid.df")
    
    meta = MetaModel(problem="regression", db=tinydb)
    meta.default_models()
    models = meta.models
    fit_models = fit_model.map(
        model=models,
        train_data=unmapped(transformed_train_df),
        target=unmapped(transformed_train_target),
        problem=unmapped(problem),
    )
    predict_models = predict_model.map(
        model=fit_models, valid_data=unmapped(transformed_valid_df),
    )

In [6]:
#flow.visualize(flow_state=flow_state)

In [86]:
from tinydb import TinyDB, Query
db = TinyDB("db.json")
db.all()

[]

In [ ]:
q = Query()
r = db.search(q.chunk == "svdname")

In [75]:
dir(q.chunk

['__and__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__invert__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__or__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_generate_test',
 '_path',
 '_prepare_test',
 '_test',
 'all',
 'any',
 'exists',
 'hashval',
 'matches',
 'one_of',
 'search',
 'test']

In [119]:
flow._sorted_tasks()[38]

typing.Any

In [110]:
dir(flow._sorted_tasks()[38])

['__add__',
 '__and__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__mifflin__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__new__',
 '__or__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rfloordiv__',
 '__rmod__',
 '__rmul__',
 '__ror__',
 '__rpow__',
 '__rsub__',
 '__rtruediv__',
 '__setattr__',
 '__sizeof__',
 '__slotnames__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 '__weakref__',
 'auto_generated',
 'bind',
 'cache_for',
 'cache_key',
 'cache_validator',
 'checkpoint',
 'copy',
 'inputs',
 'is_equal',
 'is_not_equal',
 'log_stdout',
 'logger',
 'map',
 'max_retries',
 'name',
 'not_',
 'or_',
 'outputs',
 'result_handler',
 'retry_delay',
 'run',
 'serialize',
 'set_dependencies',
 'set_